### Using the map matching with shape to clean the dataset
I can use the shape matching for:

- calculate speed
- reduce number of register by:
    - eliminating traces which are located in the final/start bus terminal (by shape sequence)
    - eliminating traces with GPS error (too far from route)
    - identifing where a bus trip start and ends (by shape sequence)
- identify circular lines
- identify bus driving backwards


O que posso fazer com o shape matching:

- calcular velocidades
- reduzir numero de registros do trace:
    - eliminar registros do trace que estao localizados dentro dos terminais de chegada/saída (shape sequence)
    - eliminar traces que estao com GPS error
    - identificar onde viagem do onibus começa e termina
- identificar linha circulares
- identificar se o ônibus está dirigindo para trás

In [2]:
# Como calcular as métricas de conectividade
1- Particionar os dados por Dia / hora /min /região --> 08/19/36/pompeia
2- Computar distância para registros de cada partição.
    Se distancia < 100m, criar conectividade
Será montado um grafo com pesos
quais regioes há mais conexoes
O numero de conexoes será o numero de contatos do onibus

SyntaxError: invalid syntax (<ipython-input-2-50561774a39d>, line 2)